In [1]:
#Problem 1

import numpy as np
from scipy import linalg as la
import math

def drazin_check(A, Ad, k):
    
    check = False 
    count = 0
    
    #Check 1
    
    if np.allclose(A @ Ad, Ad @ A) == True:
        count += 1
    
    #Check 2

    if np.allclose( np.linalg.matrix_power(A, k+1) @ Ad, np.linalg.matrix_power(A, k) ) == True:
        count += 1
    
    #Check 3
    
    if np.allclose(Ad @ A @ Ad, Ad) == True:
        count += 1    
    
    if count == 3:
        check = True 
    
    return check

A = np.array(([1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]))
Ad = np.array(([1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]))

check = drazin_check(A, Ad, 1)
print(check)

B = np.array(([1,1,3],[5,2,6],[-2,-1,-3]))
Bd = np.zeros((3,3))
check2 = drazin_check(B, Bd, 3)
print(check2)

C = np.array(([5,2,3],[5,2,6],[-2,-1,-3]))
Cd = np.array(([5,2,7],[5,2,8],[-2,-1,-3]))
check3 = drazin_check(C, Cd, 5)
print(check3)

True
True
False


In [2]:
#Problem 2

def drazin (A, tol):
    
    n, n = np.shape(A)
    f = lambda x: abs(x) > tol
    Q1, S, k1 = la.schur(A, sort = f)
    g = lambda x: abs(x) <= tol
    Q2, T, k2 = la.schur(A, sort = g)
    
    U = np.hstack((S[:,:k1],T[:,:(n-k1)]))
    Uinv = la.inv(U)
    V = Uinv @ A @ U
    Z = np.zeros((n,n))
    
    if k1 != 0:
        Minv = la.inv(V[:k1,:k1])
        Z[:k1,:k1] = Minv
    
    return U @ Z @ Uinv

Ad_new = drazin(A, 0.01)

print(Ad_new)

check = drazin_check(A, Ad_new, 1)
print(check)

[[  1.  -3.   9.  81.]
 [  0.   1.  -3. -18.]
 [  0.   0.   1.   3.]
 [  0.   0.   0.   0.]]
True


In [3]:
#Problem 3

def getr(A):
    
    m, n = np.shape(A)
    totals = np.sum(A, 0)
    D = np.diag(totals)
    
    L = D - A
    
    R = np.zeros((m,n))
    I = np.identity((m))
    
    for j in range (0, np.shape(R)[0]):
        for i in range (0, np.shape(R)[1]):
            if i != j: 
                Lt = np.copy(L)
                Lt[j,:]= I[j,:]
                Ltd = drazin(Lt, 0.01)
                R[i,j] = Ltd[i,i]
    return R

ad1 = np.array(([0,1,0,0],[1,0,1,0],[0,1,0,1],[0,0,1,0],))
ad2 = np.array(([0,1],[1,0]))
ad3 = np.array(([0,1,1],[1,0,1],[1,1,0]))
ad4 = np.array(([0,3],[3,0]))
ad5 = np.array(([0,2],[2,0]))
ad6 = np.array(([0,4],[4,0]))

R1 = getr(ad1)
R2 = getr(ad2)
R3 = getr(ad3)
R4 = getr(ad4)
R5 = getr(ad5)
R6 = getr(ad6)

print(R1)
print(R2)
print(R3)
print(R4)
print(R5)
print(R6)

[[ 0.  1.  2.  3.]
 [ 1.  0.  1.  2.]
 [ 2.  1.  0.  1.]
 [ 3.  2.  1.  0.]]
[[ 0.  1.]
 [ 1.  0.]]
[[ 0.          0.66666667  0.66666667]
 [ 0.66666667  0.          0.66666667]
 [ 0.66666667  0.66666667  0.        ]]
[[ 0.          0.33333333]
 [ 0.33333333  0.        ]]
[[ 0.   0.5]
 [ 0.5  0. ]]
[[ 0.    0.25]
 [ 0.25  0.  ]]


In [14]:
#Problem 4

import pandas as pd
import numpy as np

class LinkPredictor:
    """Predict links between nodes of a network."""

    def __init__(self, filename='social_network.csv'):
        
        self.filename = filename
        self.names_pd = pd.read_csv(filename, delimiter =",", header = None)
        self.names = (self.names_pd).as_matrix(columns=None)
        self.nodes = np.zeros_like(self.names)
        self.namelist = np.array(([self.names[0,0]], [self.names[0,1]]))
        self.name_check = ''
        
        #Give each name an index

        for i in range (0, np.shape(self.names)[0]):
            
            for j in range (0, np.shape(self.names)[1]):
                
                self.name_check = self.names[i,j]
                
                if np.all(self.namelist != self.name_check):
                    self.namelist = np.append(self.namelist, self.name_check)
        
        self.index = np.arange(0, len(self.namelist))
        
        #Replace each name by an index number 
        
        m, n = np.shape(self.names)
        
        self.name_indices = np.zeros_like(self.names)
        
        for i in range (0, np.shape(self.names)[0]):
            
            for j in range (0, np.shape(self.names)[1]):
                
                self.name_check = self.names[i,j]
                
                for k in range (0, len(self.namelist)):
                
                    if self.name_check == self.namelist[k]:
                        
                        self.name_indices[i,j] = self.index[k]   
        
        #Create adjacency matrix
        
        self.admat = np.zeros((len(self.namelist), len(self.namelist)))
        
        for a in range (0, np.shape(self.name_indices)[0]):
            
            i = self.name_indices[a, 0]
            j = self.name_indices[a, 1]
            
            self.admat[i,j] += 1
            self.admat[j,i] += 1
        
        #Effective resistance matrix
        
        self.efmat = getr(self.admat)
        
        print(self.efmat)
        
        print(np.allclose(self.efmat, (self.efmat).T))
        
LinkPredictor()

[[ 0.          0.18998381  0.19809478 ...,  0.72006729  0.72006729
   0.2333001 ]
 [ 0.18998381  0.          0.22975655 ...,  0.76301021  0.76301021
   0.27562492]
 [ 0.19809478  0.22975655  0.         ...,  0.69073556  0.69073556
   0.20953544]
 ..., 
 [ 0.72006729  0.76301021  0.69073556 ...,  0.          1.          0.53555306]
 [ 0.72006729  0.76301021  0.69073556 ...,  1.          0.          0.53555306]
 [ 0.2333001   0.27562492  0.20953544 ...,  0.53555306  0.53555306  0.        ]]
True
